<center><a href="https://www.nvidia.cn/training/"><img src="https://dli-lms.s3.amazonaws.com/assets/general/DLI_Header_White.png" width="400" height="186" /></a></center>

<br>

# <font color="#76b900">**Notebook 8 [评估]：** RAG 评估</font>

<br>

欢迎学习本课程的最后一个 notebook！您已经在前面的 notebook 中将向量存储解决方案集成到 RAG 工作流中了！您将在本 notebook 采用相同的工作流，并通过 LLM-as-a-Judge 量化地评估 RAG！

<br> 

### **学习目标：**

* 了解如何集成之前 notebook 中的技术，量化 RAG 工作流的效果。
* **最终练习**：***在课程环境中执行本 notebook，*您就能提交代码！**

<br>  

### **思考问题：**

* 在学习的过程中，请记住我们的指标代表的到底是什么。我们的工作流应该达到这些目标么？通过 LLM 进行评判是否足以评估工作流？具体的某个指标对我们的用例是否有意义？
* 如果我们在链中保留 vectorstore-as-a-memory 组件，还能通过评估么？此外，评估是否有助于评估 vectorstore-as-a-memory 的性能？

<br>  

### **Notebook 版权声明：**

* 本 notebook 是 [**NVIDIA 深度学习培训中心**](https://www.nvidia.cn/training/)的课程[**《构建大语言模型 RAG 智能体》**](https://www.nvidia.cn/training/instructor-led-workshops/building-rag-agents-with-llms/)中的一部分，未经 NVIDIA 授权不得分发。

<br> 

### **环境设置：**

In [ ]:
# %pip install -q langchain langchain-nvidia-ai-endpoints gradio rich
# %pip install -q arxiv pymupdf faiss-cpu ragas

## If you encounter a typing-extensions issue, restart your runtime and try again
# from langchain_nvidia_ai_endpoints import ChatNVIDIA
# ChatNVIDIA.get_available_models()

from functools import partial
from rich.console import Console
from rich.style import Style
from rich.theme import Theme

console = Console()
base_style = Style(color="#76B900", bold=True)
norm_style = Style(bold=True)
pprint = partial(console.print, style=base_style)
pprint2 = partial(console.print, style=norm_style)

from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings

# NVIDIAEmbeddings.get_available_models()
embedder = NVIDIAEmbeddings(model="nvidia/nv-embed-v1", truncate="END")

# ChatNVIDIA.get_available_models(base_url="http://llm_client:9000/v1")
instruct_llm = ChatNVIDIA(model="meta/llama-3.1-8b-instruct")

----

<br>

## **第 1 部分：** 预发行评估

在之前的 Notebook 中，我们成功地结合了多个概念创建文档聊天机器人，以实现高响应、有信息量的交互。然而，用户交互的多样性要求我们进行相对全面的测试，才能真正了解聊天机器人的性能。在不同场景下进行全面测试对于保障系统的功能、通用性及符合用户期望是至关重要的。

在定义好聊天机器人的角色并实现了必要的功能之后，可以分多个阶段来进行评估：

* **典型应用检测：**先测试与您的用例最贴近的场景。观察您的聊天机器人能否在有限的人工干预下进行可靠的对话。

	+ 此外，识别出应转给人工以检查/监督的边界或者分支情况（比如，换人工来确认交易或执行敏感操作），并执行。

* **边界情况（Edge Case）检测：**探索典型场景的边界，确认聊天机器人如何处理不常见但合理的场景。

	+ 在任何公开发布之前，请评估可能构成责任风险的关键边界条件，比如生成不当内容的可能性。
	+ 在所有的输出（甚至是输入）上实现测试好的护栏（guardrails），以限制不良交互，并将用户引导到可预测的对话流上。

* **渐进式推出（Progressive Rollout）：**向有限受众推出您的模型（先在内部推出，然后做 [A/B 测试](https://en.wikipedia.org/wiki/A/B_testing)）并实现分析功能，比如用量分析面板和反馈途径（投诉/喜欢/不喜欢/等等）。

这三个步骤中，前两个可由一个小团队或个人完成，并在开发过程中持续迭代。不幸的是这需要频繁地进行，还容易发生人为错误。**幸运的是，我们可以借助 LLM-as-a-Judge 范式（formulation）！**

*(是的，现在您可能已经不会惊讶了。就是因为 LLM 这么强，所以我们才专门做了这个课程)。*

---

<br>

## **第 2 部分：** LLM-as-a-Judge 范式

在对话式 AI 领域，用 LLM 作为评估器或“评委”已经成为一种对自然语言任务表现进行可配置自动测试的方法了：

* LLM 可以模拟一系列交互场景并生成合成数据，从而生成有针对性的输入来激发聊天机器人的一系列行为。
* 聊天机器人在合成数据上的反应/检索可由 LLM 进行评估或解析，并且可以强制输出成“通过”/“失败”、相似程度或抽取等格式。
* 许多此类结果都可以聚合成一个指标，按“通过评估的百分比”、“从数据源中检索到的相关信息量”、“余弦相似度均值”来解读。

这种使用 LLM 测试和量化聊天机器人质量的想法称为 ["LLM-as-a-Judge"](https://arxiv.org/abs/2306.05685)，能进行与人类判断高度一致的测评，还能进行微调并大规模应用。

**有几个现成的热门框架，包括：**
* [**RAGAs (RAG Assessement)**](https://docs.ragas.io/en/stable/)，这是自行评估的一个很好的起点。
* [**LangChain Evaluators**](https://python.langchain.com/docs/guides/evaluation/)，这是类似的第一方选项，具有许多可隐式构建的智能体。

比起按原样使用链，我们会进行扩展，用一个更定制话的方案进行评估。

<br>

---

## **第 3 部分：** [评估准备] 成对数据评估器（Pairwise Evaluator）

下面的练习将实现一个简化的 [LangChain 成对字符串评估器（Pairwise String Evaluator）](https://python.langchain.com/docs/guides/evaluation/examples/comparisons)。

**为评估 RAG 链做准备，我们需要：**

* 拉取文档索引（我们在上一个 notebook 中保存的）。
* 重构我们的 RAG 工作流。

**具体来说，我们将通过以下步骤实现评判范式：**

* 对 RAG 文档池采样，拿到两个文档块。
* 用这两个文档块生成一个合成的“基准”问答对。
* 用 RAG 智能体生成它自己的答案。
* 使用评委 LLM 比较这两种响应，其中，将生成的合成结果作为“标准答案”（ground-truth correct）。

**该链应该执行一个简单但功能强大的过程，可基于以下目标进行测试：**

<br>

> ***我的 RAG 链性能是否优于文档访问受限的聊天机器人。***

**这就是要用来做最终评估的系统！**要是想了解一下这个系统是怎么集成到 Autograder 中的，可以看看 [`frontend/frontend_server.py`](frontend/frontend_server.py) 里的实现。

<br>

### **任务 1：** 载入文档索引

在本练习中，您将载入之前 notebook 创建的 `docstore_index` 文件。下面这个单元应该能把它按原样加载到存储中。

In [ ]:
## Make sure you have docstore_index.tgz in your working directory
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings
from langchain_community.vectorstores import FAISS

# embedder = NVIDIAEmbeddings(model="nvidia/nv-embed-v1", truncate="END")

!tar xzvf docstore_index.tgz
docstore = FAISS.load_local("docstore_index", embedder, allow_dangerous_deserialization=True)
docs = list(docstore.docstore._dict.values())

def format_chunk(doc):
    return (
        f"Paper: {doc.metadata.get('Title', 'unknown')}"
        f"\n\nSummary: {doc.metadata.get('Summary', 'unknown')}"
        f"\n\nPage Body: {doc.page_content}"
    )

## This printout just confirms that your store has been retrieved
pprint(f"Constructed aggregate docstore with {len(docstore.docstore._dict)} chunks")
pprint(f"Sample Chunk:")
print(format_chunk(docs[len(docs)//2]))

<br>  

### **任务 2：[练习]** 载入 RAG 链

现在我们有了索引，可以重新创建上一个 notebook 里的 RAG 智能体了！

**主要的调整：**
* 为了简单起见，您可以去掉 vectorstore-as-a-memory 组件。加上它会增加开销，还会让练习变复杂。

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnableBranch
from langchain_core.runnables.passthrough import RunnableAssign
from langchain.document_transformers import LongContextReorder

from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings

from functools import partial
from operator import itemgetter

import gradio as gr

#####################################################################

# NVIDIAEmbeddings.get_available_models()
embedder = NVIDIAEmbeddings(model="nvidia/nv-embed-v1", truncate="END")

# ChatNVIDIA.get_available_models()
instruct_llm = ChatNVIDIA(model="meta/llama-3.1-8b-instruct")
llm = instruct_llm | StrOutputParser()

#####################################################################

def docs2str(docs, title="Document"):
    """Useful utility for making chunks into context string. Optional, but useful"""
    out_str = ""
    for doc in docs:
        doc_name = getattr(doc, 'metadata', {}).get('Title', title)
        if doc_name: out_str += f"[Quote from {doc_name}] "
        out_str += getattr(doc, 'page_content', str(doc)) + "\n"
    return out_str

chat_prompt = ChatPromptTemplate.from_template(
    "You are a document chatbot. Help the user as they ask questions about documents."
    " User messaged just asked you a question: {input}\n\n"
    " The following information may be useful for your response: "
    " Document Retrieval:\n{context}\n\n"
    " (Answer only from retrieval. Only cite sources that are used. Make your response conversational)"
    "\n\nUser Question: {input}"
)

def output_puller(inputs):
    """"Output generator. Useful if your chain returns a dictionary with key 'output'"""
    if isinstance(inputs, dict):
        inputs = [inputs]
    for token in inputs:
        if token.get('output'):
            yield token.get('output')

#####################################################################
## TODO: Pull in your desired RAG Chain. Memory not necessary

## Chain 1 Specs: "Hello World" -> retrieval_chain 
##   -> {'input': <str>, 'context' : <str>}
long_reorder = RunnableLambda(LongContextReorder().transform_documents)  ## GIVEN
context_getter = RunnableLambda(lambda x: x)  ## TODO
retrieval_chain = {'input' : (lambda x: x)} | RunnableAssign({'context' : context_getter})

## Chain 2 Specs: retrieval_chain -> generator_chain 
##   -> {"output" : <str>, ...} -> output_puller
generator_chain = RunnableLambda(lambda x: x)  ## TODO
generator_chain = {'output' : generator_chain} | RunnableLambda(output_puller)  ## GIVEN

## END TODO
#####################################################################

rag_chain = retrieval_chain | generator_chain

# pprint(rag_chain.invoke("Tell me something interesting!"))
for token in rag_chain.stream("Tell me something interesting!"):
    print(token, end="")

<br>  

### **第 3 步：** 生成合成问答对

在本节中，我们可以实现评估流程的前两个步骤：

* **对 RAG 文档池采样，拿到两个文档块。**
* **用这两个文档块生成一个合成的“基准”问答对。**
* 用 RAG 智能体生成它自己的答案。
* 使用评委 LLM 比较这两种响应，其中，将生成的合成结果作为“标准答案”（ground-truth correct）。

该链应该执行一个简单但功能强大的过程，可基于以下目标进行测试：

> 我的 RAG 链性能是否优于文档访问受限的聊天机器人。

In [ ]:
import random

num_questions = 3
synth_questions = []
synth_answers = []

simple_prompt = ChatPromptTemplate.from_messages([('system', '{system}'), ('user', 'INPUT: {input}')])

for i in range(num_questions):
    doc1, doc2 = random.sample(docs, 2)
    sys_msg = (
        "Use the documents provided by the user to generate an interesting question-answer pair."
        " Try to use both documents if possible, and rely more on the document bodies than the summary."
        " Use the format:\nQuestion: (good question, 1-3 sentences, detailed)\n\nAnswer: (answer derived from the documents)"
        " DO NOT SAY: \"Here is an interesting question pair\" or similar. FOLLOW FORMAT!"
    )
    usr_msg = (
        f"Document1: {format_chunk(doc1)}\n\n"
        f"Document2: {format_chunk(doc2)}"
    )

    qa_pair = (simple_prompt | llm).invoke({'system': sys_msg, 'input': usr_msg})
    synth_questions += [qa_pair.split('\n\n')[0]]
    synth_answers += [qa_pair.split('\n\n')[1]]
    pprint2(f"QA Pair {i+1}")
    pprint2(synth_questions[-1])
    pprint(synth_answers[-1])
    print()

<br>  

### **第 4 步：** 回答合成问题

在本节中，我们可以实现评估流程的第三个步骤：

* 对 RAG 文档池采样，拿到两个文档块。
* 用这两个文档块生成一个合成的“基准”问答对。
* **用 RAG 智能体生成它自己的答案。**
* 使用评委 LLM 比较这两种响应，其中，将生成的合成结果作为“标准答案”（ground-truth correct）。

该链应该执行一个简单但功能强大的过程，可基于以下目标进行测试：

> 我的 RAG 链性能是否优于文档访问受限的聊天机器人。

In [ ]:
## TODO: Generate some synthetic answers to the questions above.
##   Try to use the same syntax as the cell above
rag_answers = []
for i, q in enumerate(synth_questions):
    ## TODO: Compute the RAG Answer
    rag_answer = ""
    rag_answers += [rag_answer]
    pprint2(f"QA Pair {i+1}", q, "", sep="\n")
    pprint(f"RAG Answer: {rag_answer}", "", sep='\n')


<br>  

### **第 5 步：** 实现人类偏好指标

在本节中，我们可以实现评估流程的第四个步骤：

* 对 RAG 文档池采样，拿到两个文档块。
* 用这两个文档块生成一个合成的“基准”问答对。
* 用 RAG 智能体生成它自己的答案。
* **使用评委 LLM 比较这两种响应，其中，将生成的合成结果作为“标准答案”（ground-truth correct）。**

该链应该执行一个简单但功能强大的过程，可基于以下目标进行测试：

> 我的 RAG 链性能是否优于文档访问受限的聊天机器人。

In [ ]:
eval_prompt = ChatPromptTemplate.from_template("""INSTRUCTION: 
Evaluate the following Question-Answer pair for human preference and consistency.
Assume the first answer is a ground truth answer and has to be correct.
Assume the second answer may or may not be true.
[1] The second answer lies, does not answer the question, or is inferior to the first answer.
[2] The second answer is better than the first and does not introduce any inconsistencies.

Output Format:
[Score] Justification

{qa_trio}

EVALUATION: 
""")

pref_score = []

trio_gen = zip(synth_questions, synth_answers, rag_answers)
for i, (q, a_synth, a_rag) in enumerate(trio_gen):
    pprint2(f"Set {i+1}\n\nQuestion: {q}\n\n")

    qa_trio = f"Question: {q}\n\nAnswer 1 (Ground Truth): {a_synth}\n\n Answer 2 (New Answer): {a_rag}"
    pref_score += [(eval_prompt | llm).invoke({'qa_trio': qa_trio})]
    pprint(f"Synth Answer: {a_synth}\n\n")
    pprint(f"RAG Answer: {a_rag}\n\n")
    pprint2(f"Synth Evaluation: {pref_score[-1]}\n\n")

<br>  

**恭喜！我们现在有了一个能评估我们工作流的 LLM 系统！**我们现在已经有了一些评判结果，可以简单地聚合出一个结果，看看在 LLM 的眼里我们的范式怎么样：

In [ ]:
pref_score = sum(("[2]" in score) for score in pref_score) / len(pref_score)
print(f"Preference Score: {pref_score}")

----

<br>

## **第 4 部分：** 高级范式

上述的练习都是在帮您为课程的最终评估做准备，展示了一个简单但有效的评估器链是怎么构建的。看过它的具体使用过程之后，您应该就能理解我们为您提供的目标和实现了。

话说回来，这个指标只是帮我们回答以下问题的：
* **哪种行为是对流水线很重要的？**
* **要展示和评估这种行为，我们需要做些什么？**

从这两个问题中，我们还能得出大量其它评估指标，这些指标可以评估不同的属性、整合不同的评估器链技术，甚至需要不同的流程组织策略。以下列出了一些其它常用的范式，尽管远算不上穷尽：

* **风格评估（Style Evaluation）：**一些简单的评估范式可能就是想弄清楚：“我来提几个问题，看看输出像不像那么回事”。这样可以根据提供给评委 LLM 的描述来确定聊天机器人是否“表现出了应有的样子”。这种评估可以仅通过一些提示工程和 while 循环就实现出来。
* **真值评估（Groud-Truth Evaluation）：**在我们的链中，我们通过采样合成生成了一些随机问题和答案，但实际上您可能已有一些有代表性的问题和答案，需要聊天机器人能始终如一地正确回答！在这种情况下，就需要对上面的练习链进行调整，并在开发过程中持续监控。
* **检索/增强评估（Retrieval/Augmentation Evaluation）：**本课程对哪种预处理和提示步骤有利于工作流作了许多假设，其中大部分是通过实验确定的。文档预处理、分块策略、模型选择和提示词等因素都发挥着重要作用，因此创建验证这些决策的指标可能会有意义。着类指标可能需要您的工作流输出上下文块（context chunks），甚至完全依赖嵌入相似度来比较。尝试实现支持多个评估策略的链时，请记住这一点。可以把 [**RagasEvaluatorChain**](https://docs.ragas.io/en/latest/howtos/integrations/langchain.html) 抽象作为制定自定义通用评估流程的良好起点。
* **轨迹评估（Trajectory Evaluation）：**使用更高级的智能体范式，您可以实现一个假设存在对话内存的多查询策略。借此，您可以实现一个能做到以下几点的评估智能体：
	+ 按顺序提出一系列问题，评估智能体在适应和迎合场景方面的能力。这种系统通常会考虑一系列对应关系，旨在评估智能体的对话“轨迹”。[**LangChain 轨迹评估文档**](https://python.langchain.com/docs/guides/evaluation/trajectory/)是一个很好的起点。
	+ 或者，您还可以实现一个评估智能体，通过与聊天机器人交互来进行评估。这样的智能体可以评估机器人是否能自然地将对话引导到问题的解决方案上，甚至可以用来生成性能的报告。[**LangChain 智能体文档**](https://python.langchain.com/docs/modules/agents/concepts) 是一个很好的起点！

<br>

最后，请务必恰当地使用工具。在课程的这个时候，您应该已经熟悉 LLM 的核心价值了：**它们功能强大、可扩展、可预测、可控且可编排 ...... 但默认情况下，它们的行为会变得不可预测。**评估您的需求，制定和验证您的工作流，尽可能多地进行控制，让您的系统能够稳定、高效地工作。

----

<br>

## **第 5 部分：[评估]** 课程评估

欢迎来到本课程的最后一个练习！希望您喜欢这个课程的内容，并准备好得分了！在这部分：

* **确保您处于课程环境中**
* **请确保 `docstore_index/` 已上传至课程环境，即完成 [`07_vectorstores.ipynb`](07_vectorstores.ipynb) 中的练习**
	+ **且至少包含一篇最近更新过的 [Arxiv 论文](https://arxiv.org/search/advanced)。**
	+ **确保没有已经在占用端口的 09_langserve.ipynb 会话。评估要求您实现新的 /retriever 和 /generator 入口！！**

**目标：** 

启动前端服务 frontend 后，您在前端 Web 界面点击右下角的“Evaluate”便会触发 [`frontend/frontend_block.py`](frontend/frontend_block.py) 里的评估代码。您的目标就是借助我们在这门课程中构建的工作流，通过评估代码设置的通过条件！请将 [`09_langserve.ipynb`](09_langserve.ipynb) 作为参考！

**提示：** 
- 可以参考 [`09_langserve.ipynb`](09_langserve.ipynb) 中已有的 `basic_chat` 服务，实现 `retriever` 和 `generator` 服务。
- 为了达到最好的效果，“Main Route”应该选择“Basic”还是“RAG”？相信您现在可以自行做出判断了。

**注意：**
- 如果您的得分略小于通过评估所需的分数，有可能是语言模型在输出格式上表现不稳定导致的，可以尝试再次运行。

现在，运行下方代码打开前端 Web 界面，点击右下角的“Evaluate”完成课程评估吧。

In [ ]:
%%js
var url = 'http://'+window.location.host+':8090';
element.innerHTML = '<a style="color:green;" target="_blank" href='+url+'><h1>< Link To Gradio Frontend ></h1></a>';

<br>

**得到通过评估的回复后**（“Congrats! You've passed the assessment!!”），不要停止课程环境，在浏览器中回到您启动课程环境的网页，单击“ASSESS TASK”按钮！

> <img src="imgs/assess.png" width=1200px/>

<br>

----

<br>

## <font color="#76b900">**恭喜您完成了课程**</font>

希望本课程既令人兴奋还具有一定的挑战性，能为您从事 LLM 和 RAG 系统开发的前沿工作做好充分的准备！现在，您应该已经具备了应对行业级挑战所需的技能，能用开源模型和框架部署 RAG 应用了。

**您可能会感兴趣的一些 NVIDIA 产品：**
* [**NVIDIA NIMs**](https://www.nvidia.com/en-us/ai/)，提供了可本地部署的微服务启动例程。
* [**TensorRT-LLM**](https://github.com/NVIDIA/TensorRT-LLM) 是目前推荐的，在生产环境部署 GPU 加速的 LLM 模型的引擎。
* [**NVIDIA 的生成式 AI 示例库**](https://github.com/NVIDIA/GenerativeAIExamples)，包括了当前的官方微服务示例应用，并将持续发布新的生产工作流。
* [**基于知识的聊天机器人技术简介**](https://resources.nvidia.com/en-us-generative-ai-chatbot-workflow/knowledge-base-chatbot-technical-brief)，探讨了更多的 RAG 系统公开信息。

**此外，您可能有兴趣深入探讨的一些重要主题：**
* [**LlamaIndex**](https://www.llamaindex.ai/)，有强大的组件，可以增强和改进 LangChain RAG 功能。
* [**LangSmith**](https://docs.smith.langchain.com/)，LangChain 提供的智能体生产化服务。
* [**Gradio**](https://www.gradio.app/)，虽然在课程中有所提及，但还有更多值得探索的接口选项。可以从 [**HuggingFace Spaces**](https://huggingface.co/spaces) 来获取一些灵感。
* [**LangGraph**](https://python.langchain.com/docs/langgraph/)，是一个基于图形的 LLM 编排框架，对于那些对[多智能体工作流](https://blog.langchain.dev/langgraph-multi-agent-workflows/)感兴趣的学员是个很好的起点。
* [**DSPy**](https://github.com/stanfordnlp/dspy)，一个流工程框架（flow engineering framework），允许您根据性能结果优化 LLM 编排流程。